In [1]:
! pip3 install --upgrade --user google-cloud-aiplatform==1.60.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 63.7 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
import requests
from vertexai.generative_models import (
    Content,
    FunctionDeclaration,
    GenerationConfig,
    GenerativeModel,
    Part,
    Tool,
)

In [17]:
# TODO: Define a function to add two numerical inputs and return the result.
# Keep the print statement within the function.
def add_func(a,b):
    result = a + b
    print("Calling add function")
    return result

# TODO: Define a function to multiply two numerical inputs and return the result.
# Keep the print statement within the function.
def multiply_func(a,b):
    result = a * b
    print("Calling multiply function")
    return result

    
# TODO: Create FunctionDeclarations for your functions
# Specify a function declaration and parameters for an API request
add_func = FunctionDeclaration(
    name="add_func",
    description="Add two numerical inputs and return the result",
    # Function parameters are specified in OpenAPI JSON schema format
    parameters={
        "type": "object",
        "properties": {
            "a": {"type": "number", "description": "The first number to add"},
            "b": {"type": "number", "description": "The second number to add"},
        },
        "required": ["a", "b"],
    },
)

multiply_func = FunctionDeclaration(
    name="multiply_func",
    description="Multiply two numerical inputs and return the result",
    # Function parameters are specified in OpenAPI JSON schema format
    parameters={
        "type": "object",
        "properties": {
            "a": {"type": "number", "description": "The first number to multiply"},
            "b": {"type": "number", "description": "The second number to multiply"},
        },
        "required": ["a", "b"],
    },
)


In [18]:
# TODO: Define a math Tool with the add and multiply functions.
math_tool = Tool(
    function_declarations=[
        add_func,
        multiply_func,
    ],
)

In [23]:
# TODO: Initialize the model with gemini-1.5-pro-001, the tools, a temperature of 0, and system instructions instructing the model to:
    # - Fulfill the user's instructions.
    # - If asked to add or multiply numbers, call the provided functions.
    # - You may call one function after the other if needed.
    # - Repeat the result to the user.
    
model = GenerativeModel(
    model_name="gemini-1.5-pro-001",
    generation_config=GenerationConfig(temperature=0),
    system_instruction=(
            "You are a helpful assistant. Fulfill the user's instructions."
            "If asked to add or multiply numbers, call the provided functions."
            "You may call one function after the other if needed. "
            "Repeat the result to the user."
        ),
    tools=[math_tool],
)

In [24]:
chat = model.start_chat()

In [25]:
def handle_response(response):
    # If there is a function call, then invoke it
    # Otherwise, print the response.
    if response.candidates[0].function_calls:
        function_call = response.candidates[0].function_calls[0]
    else:
        print(response.text)
        return

    # Check the function call and handle it
    if function_call.name == "add_func":
        # Extract the arguments to use in your function
        args = function_call.args
        a = args.get("a")
        b = args.get("b")

        # Call your add function
        result = add_func(a, b)

        # Send the result back to the chat session with the model
        response = chat.send_message(
            Part.from_function_response(
                name="add_func",
                response={
                    "result": result,
                },
            )
        )

        # Make a recursive call of this handler function
        handle_response(response)

    elif function_call.name == "multiply_func":
        # Extract the arguments to use in your function
        args = function_call.args
        a = args.get("a")
        b = args.get("b")

        # Call your multiply function
        result = multiply_func(a, b)

        # Send the result back to the chat session with the model
        response = chat.send_message(
            Part.from_function_response(
                name="multiply_func",
                response={
                    "result": result,
                },
            )
        )

        # Make a recursive call of this handler function
        handle_response(response)

    else:
        # You shouldn't end up here
        print(f"Unexpected function call: {function_call}")

In [12]:
response = chat.send_message("Tell me a joke.")
handle_response(response)

Why don't scientists trust atoms? 

Because they make up everything! 



In [28]:
response = chat.send_message("I have 7 pizzas each with 16 slices. How many slices do I have?")
handle_response(response)

TypeError: 'FunctionDeclaration' object is not callable

In [27]:
response = chat.send_message("Doug brought 3 pizzas. Andrew brought 4 pizzas. How many pizzas did they bring together?")
handle_response(response)

TypeError: 'FunctionDeclaration' object is not callable

In [ ]:
response = chat.send_message("Doug brought 3 pizzas. Andrew brought 4 pizzas. There are 16 slices per pizza. How many slices are there?")
handle_response(response)  

In [ ]:
response = chat.send_message("Doug brought 4 pizzas, but Andrew dropped 2 on the ground. How many pizzas are left?")
handle_response(response)    